In [1]:
import os
import pickle
import copy
import numpy as np
import pandas as pd
import random
import string
import tensorflow as tf
import matplotlib.pyplot as plt

C:\Users\Deovrat\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--lr', default=0.001, type=float, help='Learning rate for training')
parser.add_argument('--batch_size', default=100, type=int, help='size of Mini-batches')
parser.add_argument('--epochs', default=50, type=int, help='Number of epochs')
parser.add_argument('--init',default=1, type=int,help='the initialization method to be used - 1 for Xavier, 2 for He')
parser.add_argument('--save_dir',default='PA2Data/save_dir', type=str,help='folder where weights need to be saved')
parser.add_argument('--dropout_prob',default=0.4,type=float, help = 'Probab of keeping in Dropout')
parser.add_argument('--decode_method',default=0,type=float, help = 'Decode method')
parser.add_argument('--beam_width',default=1,type=int, help = 'Beam_width')
parser.add_argument('--train', default='PA2Data/train.csv', type=str, help='Path of the training data')
parser.add_argument('--val', default='PA2Data/valid.csv', type=str, help='Path of the validation data')
args = parser.parse_args()

learning_rate = args.lr
batch_size = args.batch_size
init1 = args.init
epochs = args.epochs
keep_probability = args.dropout_prob
save_dir = str(args.save_dir)
train_path = str(args.train)
valid_path = str(args.val)



In [3]:
data_train = pd.read_csv(train_path)
X = data_train.drop(["id"],axis=1).values
m = X.shape[0]
X_train = X
#for i in range(X.shape[0]):
 #   X_train[i][0] = X[i][0].replace(' ','')
  #  X_train[i][1] = X[i][1].replace(' ','')


data_valid = pd.read_csv(valid_path)
X = data_valid.drop(["id"],axis=1).values
m = X.shape[0]
X_valid = X
#for i in range(X.shape[0]):
#    X_valid[i][0] = X[i][0].replace(' ','')
 #   X_valid[i][1] = X[i][1].replace(' ','')
    
data_test = pd.read_csv(test_path)
X = data_test.drop(["id"],axis=1).values
m = X.shape[0]
X_test = X
#for i in range(X.shape[0]):
#    X_test[i][0] = X[i][0].replace(' ','')

In [4]:
X_test.shape

(1000, 1)

In [5]:
list1 = []
list2 = []
for c in range(X_train.shape[0]):
    list1.append(len(X_train[c][0]))
    list2.append(len(X_train[c][1]))
X_MAX_LENGTH = max(list1)
Y_MAX_LENGTH = max(list2)
print(X_MAX_LENGTH)
print(Y_MAX_LENGTH)

121
123


In [6]:
english_char_list = []
hindi_char_list = []
for i in range(X_train.shape[0]):
    english_char_list.extend(list(X_train[i][0]))
    hindi_char_list.extend(list(X_train[i][1]))
english_vocab = list(set(english_char_list+ ['<UNK>']+ ['<GO>']+ ['<EOS>']+ ['<PAD>']))
hindi_vocab = list(set(hindi_char_list+ ['<UNK>']+ ['<GO>']+ ['<EOS>']+ ['<PAD>']))

In [7]:
vocab = list(set(english_vocab + hindi_vocab + []))

In [8]:
x_idx_to_word = [word for idx, word in enumerate(english_vocab)]


y_idx_to_word = [word for idx, word in enumerate(hindi_vocab)]


x_word_to_idx = {word:ix for ix, word in enumerate(x_idx_to_word)}
y_word_to_idx = {word: ix for ix, word in enumerate(y_idx_to_word)}

In [9]:
"""
    1st, 2nd args: raw string text to be converted
    3rd, 4th args: lookup tables for 1st and 2nd args respectively

    return: A tuple of lists (source_id_text, target_id_text) converted
"""
# empty list of converted sentences
source_text_id = []
target_text_id = []
source_text_id_valid = []
target_text_id_valid = []

# make a list of sentences (extraction)
#source_ = source_text.split("\n")
#target_sentences = target_text.split("\n")

# iterating through each sentences (# of sentences in source&target is the same)
for i in range(X_train.shape[0]):
    # extract sentences one by one
    source_word = X_train[i][0]
    target_word = X_train[i][1]

    # make a list of tokens/words (extraction) from the chosen sentence
    source_tokens = source_word.split(" ")
    target_tokens = target_word.split(" ")

    # empty list of converted words to index in the chosen sentence
    source_token_id = []
    target_token_id = []

    for index, token in enumerate(source_tokens):
        if (token != ""):
            source_token_id.append(x_word_to_idx[token])

    for index, token in enumerate(target_tokens):
        if (token != ""):
            target_token_id.append(y_word_to_idx[token])

    # put <EOS> token at the end of the chosen target sentence
    # this token suggests when to stop creating a sequence
    target_token_id.append(y_word_to_idx['<EOS>'])

    # add each converted sentences in the final list
    source_text_id.append(source_token_id)
    target_text_id.append(target_token_id)

for i in range(X_valid.shape[0]):
    # extract sentences one by one
    source_word_valid = X_valid[i][0]
    target_word_valid = X_valid[i][1]

    # make a list of tokens/words (extraction) from the chosen sentence
    source_tokens_valid = source_word_valid.split(" ")
    target_tokens_valid = target_word_valid.split(" ")

    # empty list of converted words to index in the chosen sentence
    source_token_id_valid = []
    target_token_id_valid = []

    for index, token in enumerate(source_tokens_valid):
        if (token != ""):
            source_token_id_valid.append(x_word_to_idx[token])

    for index, token in enumerate(target_tokens_valid):
        if (token != ""):
            target_token_id_valid.append(y_word_to_idx[token])

    # put <EOS> token at the end of the chosen target sentence
    # this token suggests when to stop creating a sequence
    target_token_id_valid.append(y_word_to_idx['<EOS>'])

    # add each converted sentences in the final list
    source_text_id_valid.append(source_token_id_valid)
    target_text_id_valid.append(target_token_id_valid)

In [10]:
source_text_id_test = []
for i in range(X_test.shape[0]):
    # extract sentences one by one
    source_word_test = X_test[i][0]

    # make a list of tokens/words (extraction) from the chosen sentence
    source_tokens_test = source_word_test.split(" ")

    # empty list of converted words to index in the chosen sentence
    source_token_id_test = []

    for index, token in enumerate(source_tokens_test):
        if (token != ""):
            if token in source_tokens:
                source_token_id_test.append(x_word_to_idx[token])
            else:
                source_token_id_test.append(x_word_to_idx['<UNK>'])

    # add each converted sentences in the final list
    source_text_id_test.append(source_token_id_test)


In [11]:
np.array(source_text_id_test)[0]


[7, 7, 17, 13, 35]

In [12]:
# Save data for later use
pickle.dump((
    (source_text_id, target_text_id),
    (x_word_to_idx, y_word_to_idx),
    (x_idx_to_word, y_idx_to_word)), open('preprocess.p', 'wb'))

In [13]:
import pickle

def load_preprocess():
    with open('preprocess.p', mode='rb') as in_file:
        return pickle.load(in_file)

In [14]:
import numpy as np

(source_text_id, target_text_id), (x_word_to_idx, y_word_to_idx), (x_idx_to_word, y_idx_to_word) = load_preprocess()

In [15]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.12.0
Default GPU Device: /device:GPU:0


In [16]:
def enc_dec_model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets') 
    
    target_sequence_length = tf.placeholder(tf.int32, [None], name='target_sequence_length')
    max_target_len = tf.reduce_max(target_sequence_length)    
    
    return inputs, targets, target_sequence_length, max_target_len

In [17]:
def hyperparam_inputs():
    lr_rate = tf.placeholder(tf.float32, name='lr_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return lr_rate, keep_prob

In [18]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :return: Preprocessed target data
    """
    # get '<GO>' id
    go_id = target_vocab_to_int['<GO>']
    
    after_slice = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    after_concat = tf.concat( [tf.fill([batch_size, 1], go_id), after_slice], 1)
    
    return after_concat

In [19]:
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_vocab_size, 
                   encoding_embedding_size):
    """
    :return: tuple (RNN output, RNN state)
    """
    embed = tf.contrib.layers.embed_sequence(rnn_inputs, 
                                             vocab_size=source_vocab_size, 
                                             embed_dim=encoding_embedding_size)
    
    stacked_cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.LSTMCell(rnn_size), keep_prob) for _ in range(num_layers)])
    encoding_layer
    outputs, state = tf.nn.dynamic_rnn(stacked_cells, 
                                       embed, 
                                       dtype=tf.float32)
    return outputs, state

In [20]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a training process in decoding layer 
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
                                             output_keep_prob=keep_prob)
    
    # for only input layer
    helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, 
                                               target_sequence_length)
    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                              helper, 
                                              encoder_state, 
                                              output_layer)

    # unrolling the decoder layer
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, 
                                                      impute_finished=True, 
                                                      maximum_iterations=max_summary_length)
    return outputs

In [21]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a inference process in decoding layer 
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, 
                                             output_keep_prob=keep_prob)
    
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, 
                                                      tf.fill([batch_size], start_of_sequence_id), 
                                                      end_of_sequence_id)
    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                              helper, 
                                              encoder_state, 
                                              output_layer)
    
    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, 
                                                      impute_finished=True, 
                                                      maximum_iterations=max_target_sequence_length)
    return outputs

In [22]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    target_vocab_size = len(target_vocab_to_int)
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.LSTMCell(rnn_size) for _ in range(num_layers)])
    
    with tf.variable_scope("decode"):
        output_layer = tf.layers.Dense(target_vocab_size)
        train_output = decoding_layer_train(encoder_state, 
                                            cells, 
                                            dec_embed_input, 
                                            target_sequence_length, 
                                            max_target_sequence_length, 
                                            output_layer, 
                                            keep_prob)

    with tf.variable_scope("decode", reuse=True):
        infer_output = decoding_layer_infer(encoder_state, 
                                            cells, 
                                            dec_embeddings, 
                                            target_vocab_to_int['<GO>'], 
                                            target_vocab_to_int['<EOS>'], 
                                            max_target_sequence_length, 
                                            target_vocab_size, 
                                            output_layer,
                                            batch_size,
                                            keep_prob)

    return (train_output, infer_output)

In [23]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence model
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    enc_outputs, enc_states = encoding_layer(input_data, 
                                             rnn_size, 
                                             num_layers, 
                                             keep_prob, 
                                             source_vocab_size, 
                                             enc_embedding_size)
    
    dec_input = process_decoder_input(target_data, 
                                      target_vocab_to_int, 
                                      batch_size)
    
    train_output, infer_output = decoding_layer(dec_input,
                                               enc_states, 
                                               target_sequence_length, 
                                               max_target_sentence_length,
                                               rnn_size,
                                              num_layers,
                                              target_vocab_to_int,
                                              target_vocab_size,
                                              batch_size,
                                              keep_prob,
                                              dec_embedding_size)
    
    return train_output, infer_output

In [69]:
display_step = 100


rnn_size = 512
num_layers = 1

encoding_embedding_size = 256
decoding_embedding_size = 256



In [70]:
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = load_preprocess()
max_target_word_length = max([len(word) for word in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, target_sequence_length, max_target_sequence_length = enc_dec_model_inputs()
    lr, keep_prob = hyperparam_inputs()
    
    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)
    
    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    # https://www.tensorflow.org/api_docs/python/tf/sequence_mask
    # - Returns a mask tensor representing the first N positions of each cell.
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function - weighted softmax cross entropy
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

In [71]:
def pad_word_batch(word_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_word = max([len(word) for word in word_batch])
    return [word + [pad_int] * (max_word - len(word)) for word in word_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_word_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_word_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths

In [ ]:
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text
train_target = target_int_text
valid_source = source_text_id_valid
valid_target = source_text_id_valid
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
train_loss = []
valid_loss = []
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:
                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})

                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})
                
                loss_valid = sess.run(
                cost,
                {input_data: valid_sources_batch,
                 targets: valid_targets_batch,
                 target_sequence_length: valid_targets_lengths,
                 keep_prob: 1.0})
                
                train_loss.append(loss)
                c = 1/loss_valid
                valid_loss.append(c)

                train_acc = get_accuracy(target_batch, batch_train_logits)
                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}, Loss_vaild: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss, c))
                if loss_prev < valid_loss_avg:
                    counter += 1 
                
                if loss_prev >= valid_loss_avg:
                    counter = 0
                    saver.save(sess, save_path)

                if counter >= 5:
                    break
    
    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')
    
    plt.plot(range(len(train_loss)), train_loss, 'b', label='Training loss')  
    plt.xlabel('Epochs ',fontsize=16)
    plt.ylabel('Loss',fontsize=16)
    plt.legend()
    plt.savefig('plot10')
    plt.show()
    
    plt.plot(range(len(train_loss)), train_loss, 'b', label='Training loss')  
    plt.plot(range(len(valid_loss)), valid_loss, 'r', label='Valid loss')
    plt.xlabel('Epochs ',fontsize=16)
    plt.ylabel('Loss',fontsize=16)
    plt.legend()
    plt.savefig('plot11')
    plt.show()

Epoch   0 Batch  100/262 - Train Accuracy: 0.0335, Validation Accuracy: 0.0029, Loss: 0.8955, Loss_vaild: 0.2501
Epoch   0 Batch  200/262 - Train Accuracy: 0.0504, Validation Accuracy: 0.0014, Loss: 0.5161, Loss_vaild: 0.2384
Epoch   1 Batch  100/262 - Train Accuracy: 0.0805, Validation Accuracy: 0.0000, Loss: 0.5810, Loss_vaild: 0.2082
Epoch   1 Batch  200/262 - Train Accuracy: 0.0748, Validation Accuracy: 0.0000, Loss: 0.3450, Loss_vaild: 0.1878
Epoch   2 Batch  100/262 - Train Accuracy: 0.1059, Validation Accuracy: 0.0000, Loss: 0.4382, Loss_vaild: 0.1784
Epoch   2 Batch  200/262 - Train Accuracy: 0.0972, Validation Accuracy: 0.0000, Loss: 0.2733, Loss_vaild: 0.1715
Epoch   3 Batch  100/262 - Train Accuracy: 0.1297, Validation Accuracy: 0.0000, Loss: 0.3801, Loss_vaild: 0.1659
Epoch   3 Batch  200/262 - Train Accuracy: 0.1056, Validation Accuracy: 0.0000, Loss: 0.2257, Loss_vaild: 0.1616


In [49]:
def save_params(params):
    with open('save_dir/params3.p', 'wb') as out_file:
        pickle.dump(params, out_file)


def load_params():
    with open('save_dir/params3.p', mode='rb') as in_file:
        return pickle.load(in_file)

In [50]:
# Save parameters for checkpoint
save_params(save_path)

# Checkpoint

In [51]:
_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = load_preprocess()
load_path = load_params()

## Prediction

In [52]:
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    pad_sources_batch_test = np.array(pad_word_batch(source_text_id_test, source_vocab_to_int['<PAD>']))
    pad_source_lengths_test = []
    for source in pad_sources_batch_test:
        pad_source_lengths_test.append(len(source))
    
    translate_ids = []
    for i in range(X_test.shape[0]//batch_size):
        translate_logits = sess.run(logits, {input_data:  pad_sources_batch_test[i*batch_size:(i+1)*batch_size],
                                             target_sequence_length: pad_source_lengths_test[i*batch_size:(i+1)*batch_size],
                                             keep_prob: 1.0})
        translate_ids = translate_ids + list(translate_logits)

INFO:tensorflow:Restoring parameters from checkpoints/dev


In [53]:
predict_ids = []
for i in range(X_test.shape[0]):
    #predict_ids = predict_ids + list(translate_ids[i][:list(translate_ids[i]).index(57)])
    if 62 in translate_ids[i]:
        b = translate_ids[i][:list(translate_ids[i]).index(62)]
        predict_ids.append(str(" ".join([target_int_to_vocab[j] for j in b])))
    else:
        b = translate_ids[i]
        predict_ids.append(str(" ".join([target_int_to_vocab[j] for j in b])))


In [55]:
data1 = pd.DataFrame(data=data_test['id'])
data1['HIN'] = predict_ids
data1.to_csv('predictionPA3_1.csv',index=False)